In [34]:
import json
import random
import cv2
import matplotlib.pyplot as plt
import shutil
import os
from coco_tools import *

In [35]:
target_cnt = 2000     # target_cnt 미만인 class들을 target_cnt 근처로 개수를 맞춰줌
maximum_IoU = 0.2     # 합성할 때 배경 이미지의 객체들과 겹치는 허용 정도
low_area = 5000       # 합성할 최소 사이즈
max_area = 100000     # 합성할 최대 사이즈
max_object_in_backgound = 5
dataset_path = '../dataset/'
random.seed(10)

In [37]:
os.makedirs("../dataset/cutmix_train", exist_ok=True)

In [38]:
# train 이미지들 new_train 폴더에 복사
for img_dir in os.listdir('/opt/ml/detection/dataset/train'):
    shutil.copyfile(f"/opt/ml/detection/dataset/train/{img_dir}", f"/opt/ml/detection/dataset/cutmix_train/{img_dir}")

In [39]:
with open('../dataset/train.json', 'r') as f:
    train_json = json.load(f)

In [40]:
# train_josn의 이미지 경로 /train -> /new_train으로 바꿈
for x, i in enumerate(train_json['images']):
    train_json['images'][x]['file_name'] = train_json['images'][x]['file_name'].replace('train', 'new_train')

In [41]:
# 1000개 미만인 class들만 list로 각 클래스 별로 annotation 정보 담기
section = [[i for i in train_json['annotations'] if i['category_id'] == j] for j in range(10)]
low_quantity_class = [x for x, i in enumerate(section) if len(i) < target_cnt]
low_quantity_annotations = [[i for i in train_json['annotations'] if i['category_id'] == j] for j in low_quantity_class]
[len(i) for i in section]

[3966, 6352, 897, 936, 982, 2943, 1263, 5178, 159, 468]

In [42]:
# 이미지에 객체가 5개 이하인 이미지 
under_5_img = [i for i in range(len(train_json['images'])) if len(get_annotations(train_json, i)) <= max_object_in_backgound]

for x, low_classes in enumerate(low_quantity_annotations):
    for i in range(target_cnt // len(low_classes)):
        for low_class in low_classes:
            background_img_num = random.choice(under_5_img)
            background_anns = get_annotations(train_json, background_img_num)
            if len(background_anns) == max_object_in_backgound: 
                under_5_img.remove(background_img_num)

            background_img_path = get_file_path(train_json, background_img_num)
            background_img = read_image(dataset_path, background_img_path)
            if low_class['area'] < low_area or low_class['area'] > max_area: 
                continue
            object_img_path = get_file_path(train_json, low_class['image_id'])
            object_img = read_image(dataset_path, object_img_path)
            object_img_bbox = map(int, low_class['bbox'])

            xmin, ymin, w, h = object_img_bbox
            object_img_cut = object_img[ymin:ymin+h, xmin:xmin+w].copy()

            # 합성할 이미지와 배경이미지들의 객체의 IoU가 다 0.2 미만일 때까지 랜덤 위치 추출
            cnt = 0
            while True:
                new_xmin, new_ymin = random.randint(0, 1025 - w - 1), random.randint(0, 1025 - h - 1)
                new_box = [new_xmin, new_ymin, new_xmin+w, new_ymin+h]
                ann = background_anns[0]
                check = [
                        True if IoU(new_box, [ann[0], ann[1], ann[0]+ann[2], ann[1]+ann[3]]) < maximum_IoU 
                        else False for ann in background_anns
                        ]
                cnt += 1
                if all(check) or cnt == 1000: break
            if cnt == 1000: continue

            # 추가한 부분 train_json의 annotations부분에 append
            train_json['annotations'].append({
                                "image_id": background_img_num,
                                "category_id": low_quantity_class[x],
                                "area": low_class['area'],
                                "bbox": [new_xmin, new_ymin, w, h],
                                "iscrowd": 0,
                                "id": len(train_json['annotations'])
                                })
            combine_img = background_img.copy()
            combine_img[new_ymin:new_ymin+h, new_xmin:new_xmin+w] = object_img_cut
            # print(background_img_path) # 합성한 이미지 경로 보기
            cv2.imwrite(os.path.join('../dataset/', background_img_path), combine_img)          

In [43]:
# 수정된 train_json을 new_train.json파일로 저장
with open('../dataset/new_train5.json', 'w', encoding="utf-8") as f:
    json.dump(train_json, f,ensure_ascii=False,indent="\t")

In [44]:
# 각 클래스별 개수 보기
section = [[i for i in train_json['annotations'] if i['category_id'] == j] for j in range(10)]
[len(i) for i in section]

[3966, 6352, 1887, 1862, 2290, 2943, 2025, 5178, 1827, 1268]